In [6]:
from langchain.chat_models import ChatOpenAI

chat = ChatOpenAI(
    temperature=0.1
)

In [8]:
from langchain.schema import HumanMessage, SystemMessage, AIMessage

messages = [
    SystemMessage(content="You are a a geography expert. And you only reply in Itailian."),
    AIMessage(content="Ciao, mi chiamo Paolo!"),
    HumanMessage(content="What is the distance between Rome and Naples? Also, what is your name?"),
]

chat.predict_messages(messages)



AIMessage(content='Ciao! La distanza tra Roma e Napoli è di circa 225 chilometri. Come ti chiami?')

In [13]:
from langchain.prompts import PromptTemplate, ChatPromptTemplate

template = PromptTemplate.from_template("Whate is the distance between {country_a} and {country_b}?")
prompt = template.format(country_a="Italy", country_b="France")

chat.predict(prompt)



'The distance between Italy and France varies depending on the specific locations being compared. However, the approximate distance between the two countries is around 500-600 kilometers (310-370 miles) when measuring from the northern border of Italy to the southern border of France.'

In [20]:
from langchain.prompts import PromptTemplate, ChatPromptTemplate

template = ChatPromptTemplate.from_messages([
    ("system", "You are a a geography expert. And you only reply in {language}."),
    ("ai", "Ciao, mi chiamo {name}!"),
    ("human", "What is the distance between {country_a} and {country_b}? Also, what is your name?"),
])

prompt = template.format_messages(language="Greek", country_a="Italy", country_b="France", name="Paolo")


chat.predict_messages(prompt)


AIMessage(content='Καλησπέρα, το όνομά μου είναι Παναγιώτης. Η απόσταση μεταξύ Ιταλίας και Γαλλίας είναι περίπου 1000 χιλιόμετρα.')

In [22]:
from langchain.schema import BaseOutputParser

class CommaOutputParser(BaseOutputParser):
    def parse(self, text: str) -> str:
        items = text.strip().split(",")
        return [item.strip() for item in items]

parser = CommaOutputParser()
parser.parse("Hello, world!")




['Hello', 'world!']

In [31]:
from langchain.prompts import ChatPromptTemplate

template = ChatPromptTemplate.from_messages([
    ("system", "You are a list generating machine. Everything you are asked will be answered with a comma separated list of max {max_items} in lower case. Do NOT reply with anything else."),
    ("human", "{question}"),
])

prompt = template.format_messages(question="What are the colors??", max_items=10)
result = chat.predict_messages(prompt)

parser = CommaOutputParser()
parser.parse(result.content)


['red',
 'blue',
 'green',
 'yellow',
 'orange',
 'purple',
 'pink',
 'black',
 'white',
 'brown']

In [34]:
chain = template | chat | CommaOutputParser()

chain.invoke({"question": "What are the colors?", "max_items": 10})


['red',
 'blue',
 'green',
 'yellow',
 'orange',
 'purple',
 'pink',
 'black',
 'white',
 'brown']

In [44]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()]
)

writing_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are specialized in writing haikus about programming languages. Always generate responses in the form of a haiku (5-7-5 syllable structure) and ensure each haiku centers around themes of programming, coding languages, or software development. Be concise, creative, and poetic."),
    ("human", "{language}"),
])

writing_chain = writing_prompt | chat 

reading_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are specialized in explaining haikus. When given a haiku, analyze its structure (syllable pattern, imagery, seasonal or thematic elements) and explain its meaning in a clear, thoughtful way in Korean. Focus on uncovering the poetic devices, cultural context, and emotional nuance. Be concise, insightful, and accessible to readers."),
    ("human", "{haiku}"),
])

reading_chain = reading_prompt | chat

final_chain = {"haiku": writing_chain} | reading_chain

final_chain.invoke({"language": "nodejs"})



Event-driven, fast,
JavaScript on server side,
Node.js takes flight.이 해구는 5-7-5 소절 구조를 가지고 있습니다. "이벤트 중심, 빠른," "서버 측 자바스크립트," "Node.js가 비행"이라는 세 줄로 구성되어 있습니다. 

해당 해구는 기술적인 주제를 다루고 있습니다. "이벤트 중심"과 "빠른"이라는 단어는 Node.js의 주요 특징을 강조하며, "서버 측 자바스크립트"는 Node.js가 서버 측에서 자바스크립트를 실행할 수 있는 능력을 나타냅니다. 마지막 줄인 "Node.js가 비행"은 Node.js가 혁신적이고 발전적인 기술이라는 의미를 내포하고 있습니다. 

이 해구는 기술적인 혁신과 발전에 대한 열정을 담고 있으며, Node.js의 특징과 장점을 강조하고 있습니다. 이를 통해 자바스크립트 개발자들에게 Node.js의 중요성과 가치를 상기시키는 역할을 합니다.

AIMessageChunk(content='이 해구는 5-7-5 소절 구조를 가지고 있습니다. "이벤트 중심, 빠른," "서버 측 자바스크립트," "Node.js가 비행"이라는 세 줄로 구성되어 있습니다. \n\n해당 해구는 기술적인 주제를 다루고 있습니다. "이벤트 중심"과 "빠른"이라는 단어는 Node.js의 주요 특징을 강조하며, "서버 측 자바스크립트"는 Node.js가 서버 측에서 자바스크립트를 실행할 수 있는 능력을 나타냅니다. 마지막 줄인 "Node.js가 비행"은 Node.js가 혁신적이고 발전적인 기술이라는 의미를 내포하고 있습니다. \n\n이 해구는 기술적인 혁신과 발전에 대한 열정을 담고 있으며, Node.js의 특징과 장점을 강조하고 있습니다. 이를 통해 자바스크립트 개발자들에게 Node.js의 중요성과 가치를 상기시키는 역할을 합니다.')